## Import Neccessary Libraries

In [ ]:
from transformer import *
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

## Define device that you want to use (GPU/CPU)

In [ ]:
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Connect to kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!mv kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

## Load Dataset

In [ ]:
!kaggle datasets download -d parvmodi/english-to-hindi-machine-translation-dataset

Dataset URL: https://www.kaggle.com/datasets/parvmodi/english-to-hindi-machine-translation-dataset
License(s): CC0-1.0
100% 1.04G/1.04G [00:55<00:00, 23.0MB/s]
100% 1.04G/1.04G [00:55<00:00, 20.2MB/s]


In [ ]:
!unzip '*.zip' -d /content/

Archive:  english-to-kannada-machine-translation-dataset.zip
replace /content/train.en? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/train.en       
replace /content/train.kn? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/train.kn       

Archive:  english-to-hindi-machine-translation-dataset.zip
replace /content/train.en? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/train.en       y

  inflating: /content/train.hi       y


2 archives were successfully processed.


## Text Preprocessing

In [ ]:
with open("train.en", 'r') as file:
    english_sentences = file.readlines()
with open("train.hi", 'r') as file:
    hindi_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
hindi_sentences = hindi_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]

In [ ]:
english_sentences[:5], hindi_sentences[:5]

(["however, paes, who was partnering australia's paul hanley, could only go as far as the quarterfinals where they lost to bhupathi and knowles",
  'whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.',
  'the value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.',
  'mithali to anchor indian team against australia in odis',
  'after the assent of the honble president on 8thseptember, 2016, the 101thconstitutional amendment act, 2016 came into existence'],
 ['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।',
  'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है',
  'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद

In [ ]:
vocab_en = set(' '.join(english_sentences))
vocab_hi = set(' '.join(hindi_sentences))
print(f"English Vocabulary: {len(vocab_en)}")
print(f"Kannada Vocabulary: {len(vocab_hi)}")

English Vocabulary: 68
Kannada Vocabulary: 670


In [ ]:
print(vocab_en)

{',', 'o', '5', 'u', '"', "'", 'y', 'd', ':', 'z', '>', '_', 'c', '0', ' ', 'f', '(', 'p', '<', '6', '`', '#', 'e', 'm', '.', 'a', '2', 'q', 'b', 'r', '%', '/', 'k', '-', '*', '}', '^', '=', 'v', 'i', 'l', '7', '&', 'g', 't', '|', '$', ']', '9', '+', 'x', '!', '[', '@', 's', '\\', 'w', '?', '{', 'n', '4', '~', '8', 'j', 'h', ')', '1', '3'}


In [ ]:
print(vocab_hi)

{'य़', 'व', 'β', 'ត', '˝', '•', 'פ', 'ड़', '♦', 'ू', '६', 'ន', 'ර', 'ई', '¡', 'c', 'घ', '।', 'ऐ', 'হ', '*', 'រ', 'E', 'g', 'æ', '|', 'ª', '❑', 'ன', 'ó', 'ਆ', 'ֶ', '관', 'ø', 'ण', 'š', '९', 'ग़', 'Ó', 'ঙ', '►', 'ঘ', 'ច', 'ُ', 'ப', 'إ', 'ர', '°', '§', 'd', '▫', '\u200e', '0', 'א', '□', 'দ', '\ue208', '…', 'Ṭ', 'ल', 'ढ़', 'ॐ', '#', 'ऒ', 'ප', 'ی', 'Ω', 'レ', '⊆', 'н', '្', 'נ', '\u200b', 'ز', '্', 'ō', 'ʊ', 'म', '\\', 'ष', 'ǔ', '~', 'प', '鲁', 'Ć', 'ش', 'ế', 'ḍ', 'ক', 'ֵ', 'ೆ', 'J', '਼', 'গ', '_', 'ك', 'ع', 'L', 'ʁ', 'с', 'স', '%', '॥', 'ٓ', 't', 'Ч', '่', 'ट', 'D', 'ে', 'ु', '¬', 'ऋ', '西', 'ॉ', '\u2061', '׃', 'ف', 'ē', 'C', 'ত', 'B', 'ा', 'ऩ', ':', 'ল', 'ி', 'ᠠ', 'f', 'ہ', 'س', '\uf0d8', '\uf146', '明', '州', 'ठ', '›', '國', '現', 'ා', 'ो', 'ម', '½', '’', '代', '३', '{', '™', 'ট', 'þ', 'o', 'Ⓡ', '5', 'ة', 'ৱ', 'ô', 'ν', 'Ž', 'ε', 'ृ', 'ு', 'ො', 'μ', '≈', '經', 'ر', '`', 'ऊ', 'ಯ', '☺', 'ĺ', 'ਸ', 'U', 'ಮ', '-', '&', 'Ю', 'ш', 'у', 'ò', '死', 'Ş', 'x', 'ḷ', 'ස', 'ී', '☻', '春', 'ै', 'য', 'ב', '1', 'ז', 'и'

In [ ]:
# Generated this by filtering Appendix code
START_TOKEN = '<START>'
PADDING_TOKEN = '<PAD>'
END_TOKEN = '<END>'

# Hindi Vocabulary
hindi_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?',
                      'ँ', 'ं', 'ः', 'ऄ', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऌ', 'ऍ', 'ऎ', 'ए', 'ऐ', 'ऑ', 'ऒ', 'ओ', 'औ',
                      'क', 'ख', 'ग', 'घ', 'ङ',
                      'च', 'छ', 'ज', 'झ', 'ञ',
                      'ट', 'ठ', 'ड', 'ढ', 'ण',
                      'त', 'थ', 'द', 'ध', 'न',
                      'प', 'फ', 'ब', 'भ', 'म',
                      'य', 'र', 'ऱ', 'ल', 'ळ', 'ऴ', 'व', 'श', 'ष', 'स', 'ह',
                      '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॅ', 'ॆ', 'े', 'ै', 'ॉ', 'ॊ', 'ो', 'ौ', '्',
                      'ॐ', '॑', '॒',
                      '०', '१', '२', '३', '४', '५', '६', '७', '८', '९',
                      PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [ ]:
index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [ ]:
print(index_to_english)

{0: '<START>', 1: ' ', 2: '!', 3: '"', 4: '#', 5: '$', 6: '%', 7: '&', 8: "'", 9: '(', 10: ')', 11: '*', 12: '+', 13: ',', 14: '-', 15: '.', 16: '/', 17: '0', 18: '1', 19: '2', 20: '3', 21: '4', 22: '5', 23: '6', 24: '7', 25: '8', 26: '9', 27: ':', 28: '<', 29: '=', 30: '>', 31: '?', 32: '@', 33: '[', 34: '\\', 35: ']', 36: '^', 37: '_', 38: '`', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z', 65: '{', 66: '|', 67: '}', 68: '~', 69: '<PAD>', 70: '<END>'}


In [ ]:
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Hindi: {np.percentile([len(x) for x in hindi_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

97th percentile length Hindi: 258.0
97th percentile length English: 267.0


In [ ]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(hindi_sentences)):
    hindi_sentence, english_sentence = hindi_sentences[index], english_sentences[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 200000
Number of valid sentences: 51470


In [ ]:
hindi_sentences = [hindi_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [ ]:
hindi_sentences[:5]

['और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है',
 'आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को',
 'अदालत ने इस मामले में आगे की सुनवाई के लिए एक फरवरी की तारीख़ तय की',
 'जहाँ पर ट्रैक को विभाजित किया जाना है, कृपया वह स्थान चुनें.',
 'झारखंड के मुख्यमंत्री हेमंत सोरेन (फोटोः पीटीआई)']

In [ ]:
english_sentence[:5]

'india'

In [ ]:
d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
hi_vocab_size = len(hindi_vocabulary)

transformer = Transformer(d_model=d_model,
                          ffn_hidden=ffn_hidden,
                          num_heads=num_heads,
                          drop_prob=drop_prob,
                          num_layers=num_layers,
                          max_sequence_length=max_sequence_length,
                          kn_vocab_size=hi_vocab_size,
                          english_to_index=english_to_index,
                          kannada_to_index=hindi_to_index,
                          START_TOKEN=START_TOKEN,
                          END_TOKEN=END_TOKEN,
                          PADDING_TOKEN=PADDING_TOKEN)

In [ ]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [ ]:
class TextDataset(Dataset):

    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.hindi_sentences[idx]

In [ ]:
dataset = TextDataset(english_sentences, hindi_sentences)
dataset[:5]

(['whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.',
  'mithali to anchor indian team against australia in odis',
  'the court has fixed a hearing for february 12',
  'please select the position where the track should be split.',
  'jharkhand chief minister hemant soren'],
 ['और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है',
  'आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को',
  'अदालत ने इस मामले में आगे की सुनवाई के लिए एक फरवरी की तारीख़ तय की',
  'जहाँ पर ट्रैक को विभाजित किया जाना है, कृपया वह स्थान चुनें.',
  'झारखंड के मुख्यमंत्री हेमंत सोरेन (फोटोः पीटीआई)'])

In [ ]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [ ]:
for batch_num, batch in enumerate(iterator):
    print(f"Batch {batch_num}: {batch}")  # Debugging
    if batch_num > 3:
        break

Batch 0: [('whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.', 'mithali to anchor indian team against australia in odis', 'the court has fixed a hearing for february 12', 'please select the position where the track should be split.', 'jharkhand chief minister hemant soren', '"jesus responded, as he taught in the temple, ""how is it that the scribes say that the christ is the son of david?"', 'he does this because he is angry at the alleged desecration of the indian flag', 'share videos', '"""wunderlich and his wife, petra, are both gardeners, who themselves graduated from a normal high school, which they attended """"not reluctantly,"""" dirk said."""', 'hence, we must make sure we ourselves dont become careless, not allow anyone else do so', 'mayawati, akhilesh yadav seal pact for 2019 polls: bsp-sp not to contest in amethi, raebareli', 'they praise night and day, without ever tiring.', 'im fin

In [ ]:
criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
NEG_INFTY = -1e9

def create_masks(eng_batch, hi_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(hi_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

## Training Transformer

In [ ]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 5

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in tqdm(enumerate(iterator)):
        transformer.train()
        eng_batch, hi_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hi_batch)
        optim.zero_grad()
        hi_predictions = transformer(eng_batch,
                                     hi_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hi_batch, start_token=False, end_token=True)
        loss = criterian(
            hi_predictions.view(-1, hi_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hi_batch[0]}")
            hi_sentence_predicted = torch.argmax(hi_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in hi_sentence_predicted:
              if idx == hindi_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_hindi[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")


            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions = transformer(eng_sentence,
                                          kn_sentence,
                                          encoder_self_attention_mask.to(device),
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_hindi[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break

            print(f"Evaluation translation (should we go to the mall?) : {kn_sentence}")
            print("-------------------------------------------")

Epoch 0


0it [00:00, ?it/s]

Iteration 0 : 1.795615553855896
English: whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.
Hindi Translation: और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है
Hindi Prediction: हर जि (ख़श ककपने कपान कर  काला केखिया कै कां हीहते है को ह़ुदा कि लरस केनिशा जहकप़िर  के ों को सप़ञ केज़द कै और उ़ुदा को कै का्क कर केरेे हर उे ो केखिे है


1it [00:00,  2.61it/s]

Evaluation translation (should we go to the mall?) : ('क्या आप क्या करना चाहिए?<END>',)
-------------------------------------------


...

1700it [03:12,  8.73it/s]

Iteration 1700 : 1.7286291122436523
English: i got a target on my back, though.
Hindi Translation: मैं मेरी पीठ पर एक लक्ष्य है, हालांकि मिला.
Hindi Prediction: मैं उैरे साड कर कक सि्ष्म है. 'मथां ि कैल  


1701it [03:13,  4.64it/s]

Evaluation translation (should we go to the mall?) : ('क्या आप क्या कर रहे हैं?<END>',)
-------------------------------------------


1716it [03:14,  8.81it/s]


## Inference

In [ ]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence.lower(),)
  hi_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hi_sentence)
    predictions = transformer(eng_sentence,
                              hi_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_hindi[next_token_index]
    hi_sentence = (hi_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return hi_sentence[0]

In [ ]:
translation = translate("how are you")
print(translation)

क्यों को क्यों करें<END>


In [ ]:
translation = translate("india is a nice country")
print(translation)

भारत के बाद करने की संभाव<END>


In [ ]:
torch.save(transformer, "/content/drive/MyDrive/Image Captioning/transformer_model.pth")